In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# cd "/content/drive/MyDrive/Python Practical lab/SGP"

/content/drive/MyDrive/Python Practical lab/SGP


In [1]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Data Augumentation

2024-01-05 22:17:39.268339: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
batchsize=8

In [4]:
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=0.2,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2,
    height_shift_range=0.2, validation_split=0.2)

train_data= train_datagen.flow_from_directory(r'dataset/train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical',subset='training' )

validation_data= train_datagen.flow_from_directory(r'dataset/train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical', subset='validation')

Found 1975 images belonging to 4 classes.
Found 492 images belonging to 4 classes.


In [5]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(r'dataset/test',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical')

Found 433 images belonging to 4 classes.


In [6]:
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(4,activation= 'softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False

2024-01-05 22:19:08.028618: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


87910968/87910968 [==============================] - 19s 0us/step


In [7]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 80, 80, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 39, 39, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 39, 39, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 39, 39, 32)   0           ['batch_normalization[0][0]']

In [8]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

In [9]:
checkpoint = ModelCheckpoint(r'models/model.h5',monitor='val_loss',save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', patience=7, verbose= 3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3 )

callbacks=[checkpoint,earlystop,learning_rate]

In [10]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])
model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,validation_data=validation_data,validation_steps=validation_data.samples//batchsize,callbacks=callbacks,epochs=5)

Epoch 1/5


/tmp/ipykernel_22966/1532240524.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,validation_data=validation_data,validation_steps=validation_data.samples//batchsize,callbacks=callbacks,epochs=5)


246/246 [==============================] - ETA: 0s - loss: 0.6935 - accuracy: 0.6970
Epoch 1: val_loss improved from inf to 0.48224, saving model to models/model.h5
246/246 [==============================] - 49s 184ms/step - loss: 0.6935 - accuracy: 0.6970 - val_loss: 0.4822 - val_accuracy: 0.7090 - lr: 0.0010
Epoch 2/5
246/246 [==============================] - ETA: 0s - loss: 0.4781 - accuracy: 0.7458
Epoch 2: val_loss improved from 0.48224 to 0.41740, saving model to models/model.h5
246/246 [==============================] - 43s 174ms/step - loss: 0.4781 - accuracy: 0.7458 - val_loss: 0.4174 - val_accuracy: 0.7828 - lr: 0.0010
Epoch 3/5
246/246 [==============================] - ETA: 0s - loss: 0.4202 - accuracy: 0.7717
Epoch 3: val_loss improved from 0.41740 to 0.38934, saving model to models/model.h5
246/246 [==============================] - 46s 186ms/step - loss: 0.4202 - accuracy: 0.7717 - val_loss: 0.3893 - val_accuracy: 0.7480 - lr: 0.0010
Epoch 4/5
246/246 [=================